In [9]:
import os
from pathlib import Path
from tempfile import mkdtemp
from warnings import filterwarnings
from dotenv import load_dotenv
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.llms.openai import OpenAI
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.core.node_parser import MarkdownNodeParser, HierarchicalNodeParser, SemanticSplitterNodeParser
from llama_index.readers.docling import DoclingReader
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.core.chat_engine import CondenseQuestionChatEngine
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.readers.docling import DoclingReader
from llama_index.core.postprocessor import SentenceTransformerRerank


load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

/opt/anaconda3/envs/langchain_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
#""
'''BAAI/bge-small-en-v1.5 - Beijing Academy of Artificial Intelligence (BAAI)
Sentence Embedding Model / Text Embedding Model
Specifically designed for English text.
Based on E5 architecture, which itself is a modification of the MiniLM 
(or similar lightweight Transformer) architecture optimized for embedding tasks.
~60 million parameters'''

EMBED_MODEL = HuggingFaceEmbedding("BAAI/bge-small-en-v1.5")
embed_dim = len(EMBED_MODEL.get_text_embedding("Burger"))#
print(embed_dim)

384


In [11]:
SOURCE = "Guardrail_Scenarios.pdf"

In [12]:
reader = DoclingReader()
document = reader.load_data(SOURCE) 

node_parser_mk = MarkdownNodeParser()

node_parser_semantic = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=95, embed_model=EMBED_MODEL
)

#semantic_node_parser = SemanticSplitterNodeParser()
MILVUS_URI = str(Path(mkdtemp())/ 'docling_ahtsham.db')
vector_store = MilvusVectorStore(uri=MILVUS_URI,dim=embed_dim,overwrite=True)
index = VectorStoreIndex.from_documents(
    documents=document,
    transformations=[node_parser_mk, node_parser_semantic],
    storage_context=StorageContext.from_defaults(vector_store=vector_store),
    embed_model=EMBED_MODEL,
)


2025-07-12 14:47:51,040 [DEBUG][_create_connection]: Created new connection using: 161cc7a226b64520aea23dc1a2a85a29 (async_milvus_client.py:599)


In [13]:
models = ['gpt-3.5-turbo','text-davinci-003']

llm = OpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=1024,
    frequency_penalty=0
)



In [15]:


#QUERY =  'How do you ensure software compliance (licensing) in an organization '
QUERY = "when was the last olympic held?"
query_engine = index.as_query_engine(similarity_top_k=10,llm=llm)
result = query_engine.query(QUERY)
print(f"Q: {QUERY}\nA: {result.response.strip()}")
#display([(n.text, n.metadata) for n in result.source_nodes])

Q: when was the last olympic held?
A: The last Olympic Games were held in 2021.


#### Building Guardrails using Guardrails AI

In [8]:
pip uninstall guardrails-ai
pip install "guardrails-ai>=0.4.0"

Found existing installation: guardrails-ai 0.3.1
Uninstalling guardrails-ai-0.3.1:
  Would remove:
    /opt/anaconda3/envs/langchain_env/bin/guardrails
    /opt/anaconda3/envs/langchain_env/lib/python3.10/site-packages/guardrails/*
    /opt/anaconda3/envs/langchain_env/lib/python3.10/site-packages/guardrails_ai-0.3.1.dist-info/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [ ]:
guardrails hub install hub://guardrails/competitor_check --no-install-local-models -q
guardrails hub install hub://guardrails/detect_pii --no-install-local-models -q

In [7]:
import guardrails as gd
from guardrails.hub import CompetitorCheck, DetectPII
from guardrails.integrations.llama_index import GuardrailsQueryEngine
from guardrails import OnFailAction
from pydantic import BaseModel, Field

In [8]:
class SafeResponse(BaseModel):
    response : str = Field(description="The safe and appropriate response to the user's query.",
    validators=[
            CompetitorCheck(
                competitors=["Global AI Corp", "Global AI", "GlobalAICorp"],
                on_fail=OnFailAction.REASK # If competitor is mentioned, reask the LLM
            ),
            DetectPII(
                pii_entities=["EMAIL_ADDRESS", "PHONE_NUMBER"], # Detect emails and phone numbers
                on_fail=OnFailAction.FIX # Try to fix/redact PII
            )
        ]
    )


In [16]:
guard =  gd.Guard.for_pydantic(output_class=SafeResponse)

In [18]:
guard_query_engine = GuardrailsQueryEngine(query_engine,guard=guard)


In [24]:
Query = "How old is the Roman Colloseum"
response_without_guard = query_engine.query(Query)
print(response_without_guard)

response_with_guard = guard_query_engine.query(Query)
print(response_with_guard)

The Roman Colosseum is around 1,943 years old.


ValidationError: Validation failed: Validation failed: None

In [26]:
test_queries = [
    "What does InnovateTech do?",
    "Who is InnovateTech's main competitor?",
    "How can I contact InnovateTech support?",
    "Tell me about InnovateTech's partnerships.",
    "What is the capital of France?"
]

for i, query_text in enumerate(test_queries):
    print(f"\nQuery {i+1}: {query_text}")
    try:
        response = guard_query_engine.query(query_text)
        
        # Access the validated output directly from the Guardrails response
        validated_output = response.response.validated_output
        print(f"Validated Response: {validated_output.response}")

    except Exception as e:
        print(f"An error occurred during query: {e}")


Query 1: What does InnovateTech do?
An error occurred during query: 'NoneType' object has no attribute 'validated_output'

Query 2: Who is InnovateTech's main competitor?
An error occurred during query: Validation failed: Validation failed: None

Query 3: How can I contact InnovateTech support?
An error occurred during query: Validation failed: Validation failed: None

Query 4: Tell me about InnovateTech's partnerships.
An error occurred during query: Validation failed: Validation failed: None

Query 5: What is the capital of France?
An error occurred during query: Validation failed: Validation failed: None
